Converts the html file to json

In [2]:
import json
from bs4 import BeautifulSoup

path = "./bookmarks.html"

with open(path, 'r', encoding='utf-8') as f:
        html = f.read()

# Parse the HTML
soup = BeautifulSoup(html, 'html.parser')

# Find all anchor tags
anchors = soup.find_all('a')

# Create a dictionary to store the anchor text and href
result = {}
for anchor in anchors:
    texts = anchor.text
    href = anchor['href']
    result[texts] = href

# Convert the dictionary to JSON
json_data = json.dumps(result, indent=4)

# write the JSON data
# with open('./bookmarks.json', 'w') as f:
#         f.write(json_data)

#  remove all data.values that don't start with http or https
data = {key:val for key, val in result.items() if val.startswith('http')}


Langchain code 

In [3]:
import os
os.environ["OPENAI_API_KEY"] = '[OPENAI_API_KEY]'

In [4]:
from url_playwright_async import PlaywrightURLLoaderAsync
urls = list(data.values())
loader = PlaywrightURLLoaderAsync(urls=urls[:5])
documents = await loader.load()

In [5]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(texts = texts, embedding=embeddings)

ImportError: cannot import name 'deprecate_ndim_indexing' from 'pandas.core.indexers' (c:\Users\af_is\miniconda3\envs\langchain\lib\site-packages\pandas\core\indexers\__init__.py)

In [10]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), memory=memory)

In [ ]:
query = "Please provide a summary of all the articles."
result = qa({"question": query})
result["answer"]

In [ ]:
query = """
You are given the content of an html file. It has an unstructured list of anchor tags which are of interest. You are to extract and convert the anchors into a json object. The json object should have the following structure: { 'The text of the anchor': 'The href of the anchor' }.  

For example, given the following html:

``` html 
<DL>
    <p>
        <DT><A HREF="https://fs.blog/mental-models/" ADD_DATE="1617029266"
                ICON="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8..=">Mental
                Models: The Best Way to Make Intelligent Decisions (109 Models
                Explained)</A>
        <DT><A HREF="https://fs.blog/2015/07/regression-to-the-mean/"
                ADD_DATE="1617029266"
                ICON="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8..=">Regression
                Toward the Mean: An Introduction with Examples</A>
        <DT><A HREF="https://fs.blog/2015/11/map-and-territory/"
                ADD_DATE="1617029266"
                ICON="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAABAAAAAQCAYAAAAf8..=">The
                Map Is Not the Territory</A>
</DL>
```

The json object should be:
```json
{
    'Mental Models: The Best Way to Make Intelligent Decisions (109 Models Explained)' : 'https://fs.blog/mental-models/',
    'Regression Toward the Mean: An Introduction with Examples' : 'https://fs.blog/2015/07/regression-to-the-mean/',
    'The Map Is Not the Territory' : 'https://fs.blog/2015/11/map-and-territory/'
}
```

As a test, please do so the 10 anchors. If you pass the test, I will response with continue.

"""
query_bookmarks_file(query)

In [ ]:
# chain = SimpleSequentialChain(chains=[query_bookmarks_file], verbose=True)
# # chain.add_chain(query_bookmarks_file)

# # chain.run(query)

# # generate output
# output = chain.generate_output()

# # write output to file
# with open("output.json", "w") as f:
#     chain.write_output_to_file(output, f)